In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau
import pickle

# Load your dataset
data = pd.read_csv('transaction.csv')

# Parse the ISO datetime format and extract features
data['Dt'] = pd.to_datetime(data['Dt'], format='%Y-%m-%dT%H:%M:%S.%fZ')

# Extract time-based features
data['Year'] = data['Dt'].dt.year
data['Month'] = data['Dt'].dt.month
data['Day'] = data['Dt'].dt.day
data['Hour'] = data['Dt'].dt.hour
data['Minute'] = data['Dt'].dt.minute
data['Second'] = data['Dt'].dt.second
data['DayOfWeek'] = data['Dt'].dt.dayofweek  # Monday=0, Sunday=6

# You can also create additional features such as 'is_weekend'
data['IsWeekend'] = data['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

# Aggregate the TransactionCount by Hour of the Day (you can modify based on your requirement)
hourly_data = data.groupby(['Year', 'Month', 'Day', 'Hour'])['TransactionCount'].sum().reset_index()

# Sort the data by date and time
hourly_data['Dt'] = pd.to_datetime(hourly_data[['Year', 'Month', 'Day', 'Hour']])
hourly_data = hourly_data[['Dt', 'TransactionCount']].sort_values('Dt').reset_index(drop=True)

# Normalize the TransactionCount data
scaler = MinMaxScaler()
hourly_data['TransactionCount'] = scaler.fit_transform(hourly_data[['TransactionCount']])

# Set up the number of time steps for input and output (e.g., predict next hour based on last 48 hours)
TIME_STEPS = 48

def create_dataset(data, time_steps=TIME_STEPS):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(X), np.array(y)

# Prepare the dataset for training
X, y = create_dataset(hourly_data['TransactionCount'].values.reshape(-1, 1))

# Split into training and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Convert data to torch tensors
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_test_torch = torch.tensor(y_test, dtype=torch.float32)

# Model definition
class NBeatsNet(nn.Module):
    def __init__(self, input_size, output_size, num_blocks, block_layers, hidden_size, dropout=0.2):
        super(NBeatsNet, self).__init__()
        self.input_size = input_size
        self.blocks = nn.ModuleList([self.create_block(input_size, hidden_size, block_layers, dropout) for _ in range(num_blocks)])
        self.fc_out = nn.Linear(input_size, output_size)  # Final output layer to forecast future time steps

    def create_block(self, input_size, hidden_size, block_layers, dropout):
        layers = []
        for _ in range(block_layers):
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))  # Add dropout
            input_size = hidden_size  # Update input size for subsequent layers
        # This block returns both backcast (same size as input) and forecast (output size)
        backcast_layer = nn.Linear(hidden_size, self.input_size)  # To match residuals
        forecast_layer = nn.Linear(hidden_size, self.input_size)  # To predict future time steps
        return nn.ModuleDict({'block': nn.Sequential(*layers), 'backcast': backcast_layer, 'forecast': forecast_layer})

    def forward(self, x):
        residuals = x
        forecast_sum = 0
        for block in self.blocks:
            block_output = block['block'](residuals)  # Pass through block
            backcast = block['backcast'](block_output)  # Predict backcast (size should match input_size)
            forecast = block['forecast'](block_output)  # Predict forecast (size should match input_size)
            residuals = residuals - backcast  # Update residuals
            forecast_sum += forecast  # Accumulate forecasts
        return self.fc_out(forecast_sum)  # Return final accumulated forecast

# Model parameters
input_size = TIME_STEPS
output_size = 1
num_blocks = 8
block_layers = 3
hidden_size = 128

# Set seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Initialize the model, loss function, and optimizer
model = NBeatsNet(input_size, output_size, num_blocks, block_layers, hidden_size)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)

# Training loop
num_epochs = 50
batch_size = 32

def train_model(model, X_train_torch, y_train_torch, num_epochs, batch_size):
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        permutation = torch.randperm(X_train_torch.size()[0])

        for i in range(0, X_train_torch.size()[0], batch_size):
            indices = permutation[i:i + batch_size]
            batch_x, batch_y = X_train_torch[indices], y_train_torch[indices]

            # Forward pass
            output = model(batch_x)
            loss = loss_fn(output.squeeze(), batch_y)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}/{num_epochs}, Loss: {loss.item()}')

# Train the model
train_model(model, X_train_torch, y_train_torch, num_epochs, batch_size)

# Save the trained model as a .pkl file
def save_model(model, filename):
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"Model saved to {filename}")

model_filename = '/content/drive/MyDrive/Colab Notebooks/nbeats_model.pkl'
save_model(model, model_filename)

print("Training complete, model saved.")


KeyError: 'Column not found: TransactionCount'